In [1]:
# EEG Classification BCI Task
# Computational Intelligence Course Final Project
# Armin Panjehpour - 98101288

In [2]:
# Accessing google drive

from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/CI_Project_Dataset/'  


Mounted at /content/gdrive


In [88]:
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
from scipy.io import loadmat
from scipy.fft import fft

In [4]:
# Load Dataset
Data = mat = loadmat('/content/gdrive/MyDrive/Colab Notebooks/CI_Project_Dataset/CI_Project_data.mat')
print(Data.keys())
print('Dataset loaded!')

dict_keys(['__header__', '__version__', '__globals__', 'TestData', 'TrainData', 'TrainLabel'])
Dataset loaded!


In [5]:
# Training and Test Dataset
Train_Data = Data['TrainData']
Test_Data = Data['TestData']

# Training Dataset Labels
Train_Label = Data['TrainLabel']

# Dataset Size
print('Train_Data: ',Train_Data.shape)
print('Test_Data: ',Test_Data.shape)
print('Train_Label: ',Train_Label.shape)


print('Train\Test Data Created!')

Train_Data:  (30, 384, 120)
Test_Data:  (30, 384, 40)
Train_Label:  (1, 120)
Train\Test Data Created!


In [6]:
# Sampling Rate

Trial_Time = 1.5 # in seconds
Fs = Train_Data.shape[1]/Trial_Time

print('Sampling Rate is: ',Fs)

Sampling Rate is:  256.0


In [7]:
############################################ Calculate Different Features for Each Channel ############################################

In [8]:
def Var_Feature(data):
  # variance of each channel
  # dim1: channel number, dim2: time, dim3: trial number
  var = np.var(data, axis=1)
  return var

In [9]:
def amp_hist_Feature(data, n_bins, min_amp, max_amp):
  # amplitude histogram
  # dim1: channel number, dim2: time, dim3: trial number
  n_channels = data.shape[0]
  [x_selec, y_selec, z_selec] = np.asarray(np.where((np.logical_and(1 <= data, data <= 2)) == True))
  hist_vals = []
  for i in range(n_channels):
    selected_chan = np.asarray(np.where(x_selec == i))
    selected_chan_data = data[x_selec[selected_chan],y_selec[selected_chan],z_selec[selected_chan]]
    hist_selected_chan = np.histogram(selected_chan_data, n_bins)[0]
    hist_vals.append(hist_selected_chan)
  
  return np.asarray(hist_vals)

In [71]:
def AR_Coeffs(data, order):
  # autoregressive model coefficients
  # dim1: channel number, dim2: time, dim3: trial number
  n_trials = data.shape[-1]
  n_samples = data.shape[1]
  n_channels = data.shape[0]
  Y = np.zeros((n_channels, n_samples-order, n_trials))
  Y = data[:, order:n_samples, :]

  X = np.zeros((n_trials, n_channels, n_samples-order, order+1))
  X[:,:,:,0] = 1

  Coeffs = np.zeros((n_trials, n_channels, order+1))

  for i in range(n_trials):
    for j in range(n_channels):
      for k in range(n_samples-order):
        for z in range(order):
          if(k-z >= 0):
            X[i,j,k,order-z-1] = data[j,k-z,i]
          else:
            X[i,j,k,order-z-1] = 0

      a = ((X[i,j,:,:].T @ X[i,j,:,:]) + np.asarray(0.00001*np.random.random((order+1, order+1))))
      Coeffs[i,j,:] = np.linalg.inv(a) @ (X[i,j,:,:].T) @ Y[j,:,i]

  return Coeffs

In [82]:
def FF_Feature(data):
  # form factor
  # dim1: channel number, dim2: time, dim3: trial number
  signal_std = np.std(data, axis=1)
  first_deriv_Std = np.std(np.diff(data, axis=1), axis=1)
  second_deriv_Std = np.std(np.diff(np.diff(data, axis=1), axis=1), axis=1)
  FF = (second_deriv_Std/first_deriv_Std)/(first_deriv_Std/signal_std)

  return FF

In [86]:
def cov_calculator(data1, data2):
  # covariancce calculator between two vector signals
  return np.sum(np.multiply(data1-np.mean(data1),data2-np.mean(data2)))/data1.shape[0]

In [87]:
def cov_Feature(data):
  # covariance between pairs of channels
  # dim1: channel number, dim2: time, dim3: trial number

  n_trials = data.shape[-1]
  n_samples = data.shape[1]
  n_channels = data.shape[0]

  cov_matrix = np.zeros((n_trials,n_channels,n_channels))

  for i in range(n_trials):
    for j in range(n_channels):
      for k in range(n_channels):
        selected_data_ch1 = data[j,:,i]
        selected_data_ch2 = data[k,:,i]
        cov_matrix[i,j,k] = cov_calculator(selected_data_ch1,selected_data_ch2)

  return cov_matrix

In [95]:
def fft_calculator(data, Fs):
  # calculate single side band fft of a vector signal
  L = data.shape[0]
  fft_data = fft(data)
  P2 = abs(fft_data/L)
  P1 = P2[0:L/2+1]
  P1[1:-1] = 2*P1[1:-1]

  f = Fs*np.arange(0,L/2+1)/L

  return f, P1

In [92]:
def max_freq_Feature(data, Fs):
  # find the frequency with the maximum amplitude for each channel
  # dim1: channel number, dim2: time, dim3: trial number
  
  n_trials = data.shape[-1]
  n_samples = data.shape[1]
  n_channels = data.shape[0]

  max_freq = np.zeros((n_trials,n_channels))

  for i in range(n_trials):
    for j in range(n_channels):
      selected_data = data[j,:,i]
      f, max_freq[i,j] = fft_calculator(selected_data, Fs)

  return f, max_freq

In [94]:
def mean_freq_Feature(data, Fs):
  # find the normalized weighted mean of frequencies
  # dim1: channel number, dim2: time, dim3: trial number

  n_trials = data.shape[-1]
  n_samples = data.shape[1]
  n_channels = data.shape[0]

  mean_freq = np.zeros((n_trials,n_channels))

  for i in range(n_trials):
    for j in range(n_channels):
      selected_data = data[j,:,i]
      f, fft_selected_data = fft_calculator(selected_data, Fs)

      mean_freq[i,j] = np.multiply(f, fft_selected_data)/np.sum(fft_selected_data)


  return f, mean_freq

In [ ]:
############################################ Apply the Feature Functions on Training Data ############################################

In [ ]:
# calculate variance of each trail and channel
var_feature = Var_Feature(Train_Data)